In [4]:
import json
import pandas as pd

In [5]:
def export_file(filename):
    print(f'Exporting {filename}')
    with open(f'SERIAL_CAPTURE/{filename}.json') as f:
        d = json.load(f)


    def strip_data(packet, len):
        hex = packet.split(":")
        hex = hex[-len:]
        return ''.join(hex)

    data = []

    for packet in d:

        data_len = int(packet['_source']['layers']['usb']['usb.data_len'])

        packet_str = packet['_source']['layers']['ftdi-ft']

        time = packet['_source']['layers']['frame']['frame.time_relative']

        elem = {}

        if "ftdi-ft.if_a_tx_payload" in packet_str:
            #tx
            elem = {
            "time": time,
            "dir": "tx",
            "packet": strip_data(packet_str["ftdi-ft.if_a_tx_payload"], data_len),
            "len": data_len,
            }
        else:
            #rx
            elem = {
            "time": time,
            "dir": "rx",
            "packet": strip_data(packet_str["ftdi-ft.if_a_rx_payload"], data_len),
            "len": data_len,
            }

        data.append(elem)

    df = pd.DataFrame(data)

    # Create an empy dataframe with the same columns as df
    df_concat = pd.DataFrame(columns=df.columns)

    #itterate over df 

    # COMINE PACKETS
    packet = ""
    length = len(df)
    for index, row in df.iterrows():
        direction = row['dir']
        packet = packet + row['packet']


        if index == length - 1:
            # Over 2 as two chars per byte
            df_row = pd.DataFrame([[row['time'], row['dir'], packet, int(len(packet)/2)]], columns=df.columns)
            df_concat = pd.concat([df_concat, df_row])
            packet = ""
            break
            

        if df.iloc[index+1]['dir'] != direction:
            df_row = pd.DataFrame([[row['time'], row['dir'], packet, int(len(packet)/2)]], columns=df.columns)
            df_concat = pd.concat([df_concat, df_row])
            packet = ""
        

    df_concat.to_csv(f'SERIAL_CAPTURE/{filename}.csv', index=False)

In [6]:
#get list of file in directory with extension json and remove extension

# WIRESHARK FILTER -> ftdi-ft&&(ftdi-ft.if_a_rx_payload||ftdi-ft.if_a_tx_payload)
import os
files = [f.split('.')[0] for f in os.listdir('SERIAL_CAPTURE') if f.endswith('.json')]

for file in files:
    export_file(file)

Exporting errors_200
Exporting capp_press_prog_nobean
Exporting button_press_mocha
Exporting clear_errors
Exporting button_press_other
Exporting americano_prog_noerror
Exporting button_press_white_coffee
Exporting button_press_cappuccino_program_nobeans
Exporting read_errors
Exporting mach_conf
Exporting button_press_cappuccino_program_noerr_1
Exporting americano_noerror
Exporting button_press_cappuccino
Exporting americano_161_error
Exporting first_runup
Exporting button_press_hot_chocolate
Exporting button_press_americano_good_run_then_no_beans
Exporting americano_press_prog_goodrun_then_nobeans
Exporting capp_press_prog_goodrun
Exporting americano_press
Exporting button_press_latte_works
Exporting errors_200_run2
Exporting button_press_americano
Exporting bev_config
Exporting button_press_hot_water
Exporting button_press_latte
